
### GroupBy

1. Compute the **total CO₂ emissions per continent** per year by joining `emissions` with `continents`.
2. Find the **top 3 entities by average annual emissions** within each continent.
3. For each continent, calculate the **percentage share of emissions** contributed by the top country vs. the rest for the year 2000.
4. Within each continent, identify the **first and last year** emissions were reported and the difference between them.
5. For each decade, compute the **mean, min, and max emissions** per continent.

---

### Custom `.apply`

6. Write a function that returns a row’s **emission growth rate vs. the previous year** and apply it within each entity group.
7. Within each continent group, apply a function that extracts the **entity with the largest single-year emission spike**.
8. Apply a custom function to compute a **rolling 5-year average emissions** for each entity.
9. Create a custom apply that flags whether an entity’s **emissions in a given year were above or below its long-term median**.
10. For each continent, apply a function that computes a **dictionary of summary stats** (`min`, `max`, `range`, `std`) across all entities.

---

### Melt & Pivot

11. Reshape the emissions dataset so that **years become columns** and emissions are values, with one row per entity.
12. Melt the reshaped table back to **long format** and ensure it matches the original dataset.
13. Pivot the data to show a **continent × year matrix**, with total emissions as values.
14. Create a pivot table showing **average emissions per continent per decade**, then melt it back into tidy form.
15. Make a wide table with **entities as rows and continents as columns**, where each value is that entity’s total emissions over all years.

---


In [1]:
import pandas as pd 

In [ ]:
emissions = pd.read_csv('annual-co-emissions-by-region.csv')
continents = pd.read_csv('continents-according-to-our-world-in-data.csv')

In [9]:
emissions.shape
emissions.info()
emissions.describe()
emissions.columns
emissions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29137 entries, 0 to 29136
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Entity                29137 non-null  object 
 1   Code                  23497 non-null  object 
 2   Year                  29137 non-null  int64  
 3   Annual CO₂ emissions  29137 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 910.7+ KB


,Entity,Code,Year,Annual CO₂ emissions
0,Afghanistan,AFG,1949,14656.0
1,Afghanistan,AFG,1950,84272.0
2,Afghanistan,AFG,1951,91600.0
3,Afghanistan,AFG,1952,91600.0
4,Afghanistan,AFG,1953,106256.0
